# CNN手写识别
```
Author: PeterSansan
Time : 2017-4-29
```
这个例子的数据输入输出是这样变化的：
- 1）首先输入是一张28*28的图片 
- 2)第一层先通过一层卷积层，卷积层核大小为2x2，输出为32个通道（即32个特征），这样第一层卷积层输出后为[32,28,28]的数据，即可看成是32张图片  
- 3)再经过2x2模板的最大池化，输出为[32,14,14]的数据，即下采样了一次
- 4)第二层先通过一层卷积层，卷积层核大小为2x2，输出为64个通道（即64个特征），这样第二层卷积层输出为64x14x14的数据，即可看成是64张下采样后的图片
- 5）再经过2x2模板的最大池化，输出为64x7x7的数据
- 6)最后，把数据reshape展开一维数据[64x7x7]
- 7)加一层dropout层
- 8）输出为类别为10的softmax层

In [2]:
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./../mnist/MNIST_data/", one_hot=True)

Extracting ./../mnist/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./../mnist/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./../mnist/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./../mnist/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

### 卷积和池化
这部分卷积最难理解

TensorFlow在卷积和池化上有很强的灵活性。我们怎么处理边界？步长应该设多大？在这个实例里，我们会一直使用vanilla版本。我们的卷积使用1步长（stride size），0边距（padding size）的模板，保证输出和输入是同一个大小。我们的池化用简单传统的2x2大小的模板做max pooling。为了代码更简洁，我们把这部分抽象成一个函数。

In [7]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)    #这个卷积函数后面加入relu函数


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [6]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [10]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.})

Iter 1280, Minibatch Loss= 24361.654297, Training Accuracy= 0.14844
Iter 2560, Minibatch Loss= 9601.050781, Training Accuracy= 0.57031
Iter 3840, Minibatch Loss= 6754.188965, Training Accuracy= 0.68750
Iter 5120, Minibatch Loss= 4874.096680, Training Accuracy= 0.73438
Iter 6400, Minibatch Loss= 1783.806519, Training Accuracy= 0.85156
Iter 7680, Minibatch Loss= 2411.129883, Training Accuracy= 0.83594
Iter 8960, Minibatch Loss= 2502.506592, Training Accuracy= 0.85938
Iter 10240, Minibatch Loss= 2119.799072, Training Accuracy= 0.85156
Iter 11520, Minibatch Loss= 1528.163330, Training Accuracy= 0.90625
Iter 12800, Minibatch Loss= 1354.485840, Training Accuracy= 0.91406
Iter 14080, Minibatch Loss= 723.427612, Training Accuracy= 0.92188
Iter 15360, Minibatch Loss= 1198.771362, Training Accuracy= 0.90625
Iter 16640, Minibatch Loss= 1978.712646, Training Accuracy= 0.92188
Iter 17920, Minibatch Loss= 968.514526, Training Accuracy= 0.86719
Iter 19200, Minibatch Loss= 1964.318848, Training Accura

[GitHub](https://github.com/PeterSansan) , [Blog](http://petersansan.top/)